# 01 — TaskFlow: Conceitos Cloud Native e Arquitetura de Referência

**Objetivo:** dar uma visão clara do que significa *Cloud Native* e como vamos **evoluir por camadas** ao longo do curso usando a aplicação fio-condutor **TaskFlow**.

### O que é *Cloud Native*
- Mais do que um local (AWS/Azure/on‑prem), é um **modelo operacional**: elasticidade, automação, autosserviço, observabilidade.
- Pode viver **on‑premises**, em **nuvem pública** ou em **topologias híbridas** (parte on‑prem, parte pública).

### Benefícios típicos
- Escalabilidade e resiliência
- Entregas mais rápidas (agilidade)
- Custo variável e otimização de recursos
- Alinhamento com boas práticas (12‑factor, automação CI/CD, *infrastructure as code*)

### Padrões que iremos usar
- **API Gateway** (entrada única; roteamento, CORS, rate‑limit; mais tarde auth).
- **Circuit Breaker/Retry/Timeouts** (com *Resilience4j*) — introduzidos nas fases seguintes.
- **Configuração externa** (Spring Cloud Config), **Descoberta de serviços** (Eureka).
- **Observabilidade** (Actuator desde o dia 1; métricas e *tracing* nas fases finais).


### Evolução por Fases (visão de alto nível)

A formação é organizada para que **cada fase adicione uma camada**, sem apagar a anterior:

- **Fase 0** — *Hello API*: `tasks-service` (Spring Boot) com H2 e Actuator.
- **Fase 1** — **Gateway**: entrada única `/api/**` a encaminhar para o serviço.
- **Fase 2** — **Config + Discovery + Resiliência**: Config Server, Eureka, timeouts/retries.
- **Fase 3** — **Deploy** (Docker/Kubernetes) e Postgres (on-prem / cloud / híbrido).
- **Fase 4** — **Segurança + Observabilidade + CI/CD**.

O diagrama abaixo ilustra a progressão (um quadro por fase).

In [ ]:
# Diagrama das fases com matplotlib (um gráfico por fase, sem estilos/cores específicos)
import matplotlib.pyplot as plt

def draw_phase(title, boxes, arrows):
    plt.figure(figsize=(8, 3))
    ax = plt.gca()
    ax.set_axis_off()
    for (x, y, w, h, text) in boxes:
        rect = plt.Rectangle((x, y), w, h, fill=False)
        ax.add_patch(rect)
        ax.text(x + w/2, y + h/2, text, ha='center', va='center', fontsize=10, wrap=True)
    for (x1, y1, x2, y2, label) in arrows:
        ax.annotate(label if label else "", xy=(x2, y2), xytext=(x1, y1),
                    arrowprops=dict(arrowstyle="->"), ha='center', va='center', fontsize=9)
    ax.set_title(title)
    plt.show()

draw_phase(
    "Fase 0 — Hello API",
    boxes=[(0.1, 0.3, 0.25, 0.4, "Client"),
           (0.45, 0.3, 0.4, 0.4, "tasks-service (H2, Actuator)\n:8081")],
    arrows=[(0.35, 0.5, 0.45, 0.5, "")]
)

draw_phase(
    "Fase 1 — Gateway",
    boxes=[(0.05, 0.3, 0.2, 0.4, "Client"),
           (0.32, 0.3, 0.28, 0.4, "Gateway\n:8080\n(/api/tasks)"),
           (0.65, 0.3, 0.3, 0.4, "tasks-service\n:8081\n(/tasks)")],
    arrows=[(0.25, 0.5, 0.32, 0.5, ""),
            (0.60, 0.5, 0.65, 0.5, "")]
)

draw_phase(
    "Fase 2 — Config + Discovery + Resiliência",
    boxes=[(0.02, 0.3, 0.18, 0.4, "Client"),
           (0.26, 0.3, 0.24, 0.4, "Gateway\n:8080"),
           (0.52, 0.3, 0.22, 0.4, "tasks-service"),
           (0.78, 0.52, 0.2, 0.2, "Config Server"),
           (0.78, 0.28, 0.2, 0.2, "Eureka")],
    arrows=[(0.20, 0.5, 0.26, 0.5, ""),
            (0.50, 0.5, 0.52, 0.5, ""),
            (0.38, 0.64, 0.78, 0.62, "configs"),
            (0.64, 0.36, 0.78, 0.36, "register")]
)

draw_phase(
    "Fase 3 — Deploy (Docker/K8s) + Postgres",
    boxes=[(0.02, 0.3, 0.18, 0.4, "Client"),
           (0.26, 0.3, 0.22, 0.4, "Ingress/LB"),
           (0.52, 0.3, 0.22, 0.4, "Gateway"),
           (0.78, 0.4, 0.2, 0.3, "tasks-service\nPostgres")],
    arrows=[(0.20, 0.5, 0.26, 0.5, ""),
            (0.48, 0.5, 0.52, 0.5, ""),
            (0.74, 0.5, 0.78, 0.5, "")]
)

draw_phase(
    "Fase 4 — Segurança + Observabilidade + CI/CD",
    boxes=[(0.04, 0.3, 0.2, 0.4, "Client"),
           (0.30, 0.3, 0.24, 0.4, "Gateway\nAuth/Rate-limit"),
           (0.60, 0.3, 0.24, 0.4, "tasks-service"),
           (0.86, 0.52, 0.12, 0.18, "Metrics/Logs/Traces"),
           (0.86, 0.30, 0.12, 0.18, "CI/CD")],
    arrows=[(0.24, 0.5, 0.30, 0.5, ""),
            (0.54, 0.5, 0.60, 0.5, ""),
            (0.72, 0.62, 0.86, 0.61, ""),
            (0.72, 0.38, 0.86, 0.38, "")]
)


### On‑prem, Pública e Híbrida (posicionamento rápido)

- **On‑prem**: Docker/Kubernetes no datacenter; DNS/Ingress internos; segredos com Vault.
- **Pública (AWS/Azure)**: ECS/EKS/ACA/AKS; ECR/ACR; SSM/Secrets Manager/Key Vault; ALB/App Gateway.
- **Híbrida**: VPN/peering; DNS dividido; gateway na cloud e serviços/dados on‑prem (ou o inverso).

> A progressão das fases foi desenhada para que possas **ensinar/mostrar a evolução** sem reescrever o que já funciona.
